# GraphQL Advanced (Preview, Limited Functionality)

NOTE: GraphQL will be part of the 1.1 release anticipated in Q1 2025

This notebook picks up from where the last GraphQL notebook left off.

We will introduce more complex queries and will cover some example applications.

We will use the same code as in our REST_DEMO notebook to get the data we want to query.  So run the following cell to get set up.

** Remember to set the API_ENDPOINT to the correct value for your environment. **

In [1]:
# start by importing the requests library
import requests
import base64
# import the pprint library to make the output more readable
from pprint import pprint
# define the api-endpoint
# Note: API Endpoint will change to https://api.water.noaa.gov/hefs/ after testing is complete
API_ENDPOINT = "https://testing-api.water.noaa.gov/hefs/"

## Leveraging GraphQL Relay

Relay is a framework for building efficient and performant GraphQL APIs client applications. Relays are used to simplify data fetching and improve pagination and caching.

We will leave the details to the GraphQL documentation, but here are some high level points in review:
- Connection: lists of data with edges and nodes
- 
Edges:Items in the connection (hello graph theory...).- 
Nodes: Data items with unique identifiers- .
Cursorstrings that "walk" the data set, to aid in paginationo- n.
Pagination: Cursor-based navigation through d

We have seen edges, nodes, cursors and pagination -- the connections have been underneath, but it is time to "break some eggs".ata.

### Retrieving Series Data

Now we are going to revisit getting a series, but we will include its event data.

Given the underlying connections, GraphQL wise this is very simple, we add events and their underlying graph along with the series meta data.  But we need an id first:

In [2]:
# simple query for the first id
graphql_query = '''
query {
  seriesList(first: 1) {
    edges {
      node {
        id
      }
    }
  }
}
'''

# Create a dictionary to send the query
payload = {"query": graphql_query}

# Create the series request
uri = API_ENDPOINT + "/v1/graphql/"

# Perform a POST request to get the response
response = requests.post(uri, json=payload, headers=headers)

# Print the response
print("Response: ")
pprint(response.json())

# Get the first node
first_node = response.json().get('seriesList', {}).get('edges', [])[0].get('node', {}).get('id', None)

# Decode from base64 -- thanks JSON
first_node = base64.b64decode(first_node).decode('utf-8')

# Strip off series type
first_node = first_node.split(':')[1]

# Print
print("\nFirst node: " + first_node)

Response: 
{'seriesList': {'edges': []}}


IndexError: list index out of range

In [3]:
# Define the GraphQL query as a Python string
graphql_query = f'''
{{
  series(id: "{first_node}") {{
    id
    uuid
    type
    locationId
    parameterId
    ensembleId
    ensembleMemberIndex
    timeStepUnit
    timeStepMultiplier
    startDateDate
    startDateTime
    endDateDate
    endDateTime
    forecastDateDate
    forecastDateTime
    missVal
    stationName
    lat
    lon
    x
    y
    z
    units
    creationDate
    creationTime
    events {{
      edges {{
        node {{
          id
          date
          time
          value
          flag
        }}
      }}
    }}
  }}
}}
'''
# Create a dictionary to send the query
payload = {"query": graphql_query}

# Create the series request
uri = API_ENDPOINT + "/v1/graphql/"

# Perform a POST request to get the response
response = requests.post(uri, json=payload, headers=headers)

# Print the response
pprint(response.json())

NameError: name 'first_node' is not defined

In [2]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

data = response.json()

# Extracting time and value data
times = []
values = []

for edge in data['archives']['events']['edges']:
    node = edge['node']
    date_str = node['date']
    time_str = node['time']
    value = node['value']

    # Combining date and time strings into a single datetime object
    date_time_str = f"{date_str} {time_str}"
    date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

    # Adding data to lists
    times.append(date_time_obj)
    values.append(value)

# Plotting the data
plt.figure(figsize=(10, 5))
plt.plot(times, values, marker='o', linestyle='-')
plt.xlabel('Time')
plt.ylabel('CFS')
plt.title('Time vs Value Graph')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

NameError: name 'response' is not defined